In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import dotenv
dotenv.load_dotenv()

True

In [7]:
# instantiate LLM and embeddings models
llm = ChatOpenAI(model_name='gpt-4-0125-preview', temperature=0)
embeddings = OpenAIEmbeddings()

/home/mileto/dev/lab/activeloop/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# creating the documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [9]:
# create the dataset
db = Chroma(collection_name='test', persist_directory='./data', embedding_function=embeddings)

In [11]:
# add documents to the db dataset
db.add_documents(docs)

['67bbc818-c159-11ee-8b4c-00155d247e06',
 '67bbc930-c159-11ee-8b4c-00155d247e06']

In [15]:
# Creating the retriever

retrieval_qa = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=db.as_retriever()
)

In [16]:
# Creting the agent the uses the RetrievelQA chain as a tool:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
  Tool(
    name="Retrieval QA System",
    func=retrieval_qa.run,
    description="Useful for answering questions."
  )
]

agent = initialize_agent(
  tools,
  llm,
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  verbose=True
)

/home/mileto/dev/lab/activeloop/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [17]:
from IPython.display import Markdown
# Now, use the agent to ask question
response = agent.run("When was Napoleone born?")
Markdown(response)

/home/mileto/dev/lab/activeloop/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I need to find out the birth date of Napoleone, which is likely referring to Napoleon Bonaparte.
Action: Retrieval QA System
Action Input: When was Napoleon Bonaparte born?

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2



Observation: Napoleon Bonaparte was born on 15 August 1769.
Thought:I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.


Napoleon Bonaparte was born on 15 August 1769.

In [27]:
# Get the same collection of the db, to add more documents
db = Chroma(collection_name='test', persist_directory='./data', embedding_function=embeddings)

In [20]:

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
docs = text_splitter.create_documents(texts) 

In [22]:
# add documents to the db
db.add_documents(docs)


['ae5fd66a-c163-11ee-8b4c-00155d247e06',
 'ae5fd9ee-c163-11ee-8b4c-00155d247e06']

In [23]:
llm = ChatOpenAI(model='gpt-4-0125-preview', temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=db.as_retriever()
)

In [24]:
# instantiate a tool that uses the retriever
tools = [
  Tool(
    name='Retrieval QA System',
    func=retrieval_qa.run,
    description="Useful for answering questions"
  )
]

# create the agent that uses the tool
agent = initialize_agent(
  tools,
  llm,
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  verbose=True
)

In [25]:
response = agent.run("When was Michael Jordan born?")
Markdown(response)



> Entering new AgentExecutor chain...
I need to use the Retrieval QA System to find out Michael Jordan's birth date.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation: Michael Jeffrey Jordan was born on 17 February 1963.
Thought:I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.


Michael Jordan was born on 17 February 1963.

In [26]:
response = agent.run("When was Napoleone born?")
Markdown(response)



> Entering new AgentExecutor chain...
I need to find the birth date of Napoleon Bonaparte.
Action: Retrieval QA System
Action Input: When was Napoleon Bonaparte born?
Observation: Napoleon Bonaparte was born on 15 August 1769.
Thought:I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.


Napoleon Bonaparte was born on 15 August 1769.